In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QLabel, QMainWindow, QWidget, QPushButton, QFileDialog, QDesktopWidget
from PyQt5.QtGui import QPixmap, QFont
from PyQt5.QtGui import QImage
import joblib
import cv2
import numpy as np
class ImageLoaderApp(QMainWindow):
    loaded_model = joblib.load("SkinCancerDetection.joblib")
    def __init__(self):
        super().__init__()

        self.initUI()

    def initUI(self):
        
        font_size=20
        
        central_widget = QWidget(self)
        self.setCentralWidget(central_widget)
        central_widget.setStyleSheet("background-color: #d2ebfc;")
        # Get the screen dimensions
        screen = QDesktopWidget().screenGeometry()
        screenHeight = screen.height()
        screenWidth = screen.width()
        print("Screen size:", screenWidth, "x", screenHeight)
        
        self.image_title = QLabel(self)
        self.image_title.setGeometry(
            int(screenWidth * 5 / 100), int(screenHeight * 5 / 100), int(screenWidth * 21 / 100),
                         int(screenHeight * 5 / 100))
        self.image_title.setText("L'image:")
        self.image_title.setFont(QFont("HP Simplified", font_size))
        
        self.result_title = QLabel(self)
        self.result_title.setGeometry(
            int(screenWidth * 50 / 100), int(screenHeight * 5 / 100), int(screenWidth * 21 / 100),
                         int(screenHeight * 5 / 100))
        self.result_title.setText("Resultat:")
        self.result_title.setFont(QFont("HP Simplified", font_size))
        
        # Create a QLabel to display the image
        self.image_label = QLabel(self)
        self.image_label.setObjectName("image")
        self.image_label.setGeometry(
            int(screenWidth * 5 / 100), int(screenHeight * 10 / 100), 500, 500)
        self.image_label.setStyleSheet("""
            #image{
                border: 1px solid black;
                border-radius: 10px;
            }
        """)
        # Create a QLabel to display the result
        self.result_label = QLabel(self)
        self.result_label.setObjectName("result")
        self.result_label.setStyleSheet("""
                #result{
                    border-bottom:1px solid black;
                }
        """)
        self.result_label.setGeometry(
            int(screenWidth * 50 / 100), int(screenHeight * 10 / 100), int(screenWidth * 21 / 100),
                         int(screenHeight * 3 / 100))
        self.result_label.setFont(QFont("HP Simplified", 12))
        # Create a QLabel to display the result
        self.result2_label = QLabel(self)
        self.result2_label.setObjectName("result")
        self.result2_label.setStyleSheet("""
                #result{
                    border-bottom:1px solid black;
                }
        """)
        
        self.result2_label.setGeometry(
            int(screenWidth * 50 / 100), int(screenHeight * 13 / 100), int(screenWidth * 21 / 100),
                         int(screenHeight * 3 / 100))
        self.result2_label.setFont(QFont("HP Simplified", 12))
        # Create a button to open the file dialog
        self.open_button = QPushButton('Open Image', self)
        self.open_button.clicked.connect(self.open_file_dialog)

        # Create a button to predict using the SVM model
        self.predict_button = QPushButton('Predict', self)
        self.predict_button.setObjectName("predictButton")
        self.predict_button.clicked.connect(self.predict_single_image)
        self.predict_button.setGeometry(
            int(screenWidth*32/100), int(screenHeight*80/100), 130, 40)
        self.predict_button.setStyleSheet("""
                    #predictButton {
                        border: 1px solid black;
                        border-radius: 5px;
                        color: black;
                        font-weight: bold;
                    }
                    #predictButton:hover {
                        background-color: #abd8f7;
                        border: 3px solid #5dffac;
                        border-radius:15px;
                    }
                    """)

        # Set fixed sizes for buttons
        self.open_button.setObjectName("openButton")
        self.open_button.setStyleSheet("""
                    #openButton {
                        border: 1px solid black;
                        border-radius: 5px;
                        color: black;
                        font-weight: bold;
                    }
                    #openButton:hover {
                        background-color: #abd8f7;
                        border: 3px solid #5dffac;
                        border-radius:15px;
                    }
                    """)
        self.open_button.setGeometry(
            int(screenWidth*5/100), int(screenHeight*80/100), 130, 40)
        self.setGeometry(100, 100, 800, 600)
        self.setWindowTitle('Image Loader')
        self.show()

    def open_file_dialog(self):
        file_path, _ = QFileDialog.getOpenFileName(self, "Open Image", "", "Image Files (*.png *.jpg *.bmp);;All Files (*)")

        if file_path:
            self.load_and_display_image(file_path)

    def load_and_display_image(self, path):
        # Load the image using OpenCV
        image = cv2.imread(path)

        # Resize the image (e.g., to 256x256)
        resized_image = cv2.resize(image, (500, 500))
        resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
        # Create a QPixmap from the resized image
        pixmap = QPixmap(QPixmap.fromImage(QImage(resized_image.data, resized_image.shape[1], resized_image.shape[0], QImage.Format_RGB888)))

        # Set the QPixmap to the QLabel
        self.image_label.setPixmap(pixmap)

    # Save the image path for later prediction
        self.image_path = path

    def predict_single_image(self):
        if hasattr(self, 'image_path') and self.image_path:
            # Load the trained SVM model from a file
            

            # Load and preprocess the single image for prediction
            image = cv2.imread(self.image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (224, 224))
            image = image / 255.

            # Flatten the image for SVM prediction
            flattened_image = image.reshape(1, -1)
            # Get decision function values
            decision_values = self.loaded_model.decision_function(flattened_image)

            # Predict using the SVM model
            prediction = self.loaded_model.predict(flattened_image)

            # Display the prediction result
            if prediction == 0:
                result = 'Benign'
            else:
                result = 'Malignant'

            # Calculate percentage confidence
            confidence_percentage = (1 / (1 + np.exp(-abs(decision_values[0])))) * 100

            # Set the result label text with confidence
            self.result_label.setText(f"Prediction: {result}")
            self.result2_label.setText(f"Confidence: {confidence_percentage:.2f}%")
        else:
            print("Please load an image first.")

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = ImageLoaderApp()
    sys.exit(app.exec_())


Screen size: 1366 x 768
